In [36]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
#!pip install nnunetv2
#!pip install --upgrade git+https://github.com/FabianIsensee/hiddenlayer.git

In [1]:
import os
import shutil
import random
import nibabel as nib
import numpy as np
from typing import List
from nnunetv2.dataset_conversion.generate_dataset_json import generate_dataset_json

In [2]:
def gestion_nifti(source_folder, copy_folder, test_folder, mask_source, mask_copy, mask_test):

    sum_voxels = 0.0
    sum_squared_voxels = 0.0
    total_voxels = 0

    for filename in os.listdir(source_folder):
        img_path = os.path.join(source_folder, filename)
        img = nib.load(img_path)
        data = img.get_fdata()
        
        sum_voxels += np.sum(data)
        sum_squared_voxels += np.sum(data ** 2)
        total_voxels += data.size

    global_mean = sum_voxels / total_voxels
    global_std = np.sqrt((sum_squared_voxels / total_voxels) - (global_mean ** 2))
    
    for file in sorted(os.listdir(source_folder)):
        if file.endswith('.nii') or file.endswith('.nii.gz'):
            img_path = os.path.join(source_folder, file)
            img = nib.load(img_path)
            data = img.get_fdata()
            
            standardized_data = (data - global_mean) / global_std
            
            standardized_img = nib.Nifti1Image(standardized_data, img.affine, img.header)
            
            standardized_img_path = os.path.join(copy_folder, file.split('.')[0] + '.nii.gz')
            nib.save(standardized_img, standardized_img_path)

    print(f"Archivos copiados y estandarizados de {source_folder} a {copy_folder}")

    predefined_list = ['P02_T2', 'P09_T1', 'P13_T1', 'P14_T2', 'P14_T4', 'P22_T1', 'P27_T1', 'P32_T1', 'P47_T1', 'P51_T1']
    
    for file in sorted(os.listdir(copy_folder)):
        if any(name in file for name in predefined_list):
            source_path = os.path.join(copy_folder, file)
            dest_path = os.path.join(test_folder, file)
            shutil.move(source_path, dest_path)

    print(f"Archivos test movidos a {test_folder}")

    counter = 1

    for file in sorted(os.listdir(copy_folder)):
        old_path = os.path.join(copy_folder, file)
        new_name = f'BRATS_{counter:03d}_0000.nii.gz'
        new_path = os.path.join(copy_folder, new_name)
        os.rename(old_path, new_path)
        counter += 1
        
    for file in sorted(os.listdir(test_folder)):
        old_path = os.path.join(test_folder, file)
        new_name = f'BRATS_{counter:03d}_0000.nii.gz'
        new_path = os.path.join(test_folder, new_name)
        os.rename(old_path, new_path)
        counter += 1

    print(f"Archivos renombrados en {copy_folder} y {test_folder}")


    for file in sorted(os.listdir(mask_source)):
        if file.endswith('.nii') or file.endswith('.nii.gz'):
            img_path = os.path.join(mask_source, file)
            img = nib.load(img_path)
            data = img.get_fdata()
            
            compressed_img_path = os.path.join(mask_copy, file.split('.')[0] + '.nii.gz')
            compressed_img = nib.Nifti1Image(data, img.affine, img.header)
            nib.save(compressed_img, compressed_img_path)

    print(f"Archivos copiados de {mask_source} a {mask_copy}")
        
    for file in sorted(os.listdir(mask_copy)):
        if any(name in file for name in predefined_list):
            source_path = os.path.join(mask_copy, file)
            dest_path = os.path.join(mask_test, file)
            shutil.move(source_path, dest_path)

    print(f"Archivos test movidos a {mask_test}")

    counter = 1

    for file in sorted(os.listdir(mask_copy)):
        old_path = os.path.join(mask_copy, file)
        new_name = f'BRATS_{counter:03d}.nii.gz'
        new_path = os.path.join(mask_copy, new_name)
        os.rename(old_path, new_path)
        counter += 1
        
    for file in sorted(os.listdir(mask_test)):
        old_path = os.path.join(mask_test, file)
        new_name = f'BRATS_{counter:03d}.nii.gz'
        new_path = os.path.join(mask_test, new_name)
        os.rename(old_path, new_path)
        counter += 1

    print(f"Archivos renombrados en {mask_copy} y {mask_test}")


In [3]:
source_folder = "Datos/Cerebro/"
copy_folder = "nnUNet/nnUNet_raw/Dataset022_Esclerosis/imagesTr/"
test_folder = "nnUNet/nnUNet_raw/Dataset022_Esclerosis/imagesTs/"
mask_source = "Datos/Mascara/"
mask_copy = "nnUNet/nnUNet_raw/Dataset022_Esclerosis/labelsTr/"
mask_test = "nnUNet/nnUNet_raw/Dataset022_Esclerosis/labelsTs/"

if not os.listdir(copy_folder):
    gestion_nifti(source_folder, copy_folder, test_folder, mask_source, mask_copy, mask_test)
else: print('Carpetas ya llenas')

Archivos copiados y estandarizados de Datos/Cerebro/ a nnUNet/nnUNet_raw/Dataset022_Esclerosis/imagesTr/
Archivos test movidos a nnUNet/nnUNet_raw/Dataset022_Esclerosis/imagesTs/
Archivos renombrados en nnUNet/nnUNet_raw/Dataset022_Esclerosis/imagesTr/ y nnUNet/nnUNet_raw/Dataset022_Esclerosis/imagesTs/
Archivos copiados de Datos/Mascara/ a nnUNet/nnUNet_raw/Dataset022_Esclerosis/labelsTr/
Archivos test movidos a nnUNet/nnUNet_raw/Dataset022_Esclerosis/labelsTs/
Archivos renombrados en nnUNet/nnUNet_raw/Dataset022_Esclerosis/labelsTr/ y nnUNet/nnUNet_raw/Dataset022_Esclerosis/labelsTs/


In [13]:
output_folder = 'nnUNet/nnUNet_raw/Dataset022_Esclerosis/'
channel_names = {0: 'FLAIR'}
labels = {
    'background': 0,
    'lesion': 1
}
num_training_cases = 82
file_ending = '.nii.gz'
dataset_json_path = join(output_folder, 'dataset.json')
if not os.path.exists(dataset_json_path):
    generate_dataset_json(
        output_folder=output_folder,
        channel_names=channel_names,
        labels=labels,
        num_training_cases=num_training_cases,
        file_ending=file_ending,
        dataset_name='MS_FLAIR_Segmentation',
        reference='MRI segmentation dataset for multiple sclerosis lesions',
        release='1.0',
        license='CC-BY-4.0',
        description='Dataset for binary segmentation of MS lesions in FLAIR images'
    )
else:
    print(f'El archivo {dataset_json_path} ya existe. No se realizará ninguna acción.')

El archivo nnUNet/nnUNet_raw/Dataset022_Esclerosis/dataset.json ya existe. No se realizará ninguna acción.
